# Medium: Real-time Emotion Detection from Webcam using Deep Learning and OpenCV BY RAJDEEP SARKAR

In [18]:
!git status



On branch main
Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   ../.ipynb_checkpoints/Emotion Detections-checkpoint.ipynb
	deleted:    ../Emotion Detections.ipynb
	modified:   Emotion Detections.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.ipynb_checkpoints/

no changes added to commit (use "git add" and/or "git commit -a")


## 1. Import the libraries and Preprocessing


In [14]:
import cv2
import numpy as np
from tensorflow.keras.layers import Conv2D, Dropout, MaxPooling2D, Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf


In [3]:
color_mode="grayscale"

#Specify the directories holding the data/images
train_dir="train"
valid_dir="test"

#create instances of the ImageDataGenerator and rescale.
#This just means normalizing the pixels by dividing every pixel value by 255
#ImageDataGenerator is used for image augmentation/manipulation i.e normalization, random shifts and zooms
#Train_datagen is an ImageDataGenerator
train_datagen= ImageDataGenerator(rescale=1./255)
valid_datagen=ImageDataGenerator(rescale=1./255)

#Now load the data
#flow_from_directory: This method generates batches of augmented data from images in a directory. It is 
#particularly useful when you have a large dataset stored in a directory structure where subdirectories 
#represent different classes.
train_generation=train_datagen.flow_from_directory(
train_dir,#directory path
target_size=(48,48), #dimensions to which images will be resized during preprocessing
batch_size=64, #determines number of samples in each batch during training
color_mode="grayscale",
class_mode='categorical')#says that labels are one-hot encoded

valid_generation=valid_datagen.flow_from_directory(
valid_dir,
batch_size=64, 
target_size=(48,48),
color_mode="grayscale",
class_mode="categorical"
)



Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [4]:
print(type(train_datagen))

<class 'keras.src.preprocessing.image.ImageDataGenerator'>


## 2. Building the Emotion Recognition model


In [9]:
emotion_model = Sequential()#This creates an instance of a sequential model which is a linear stack of layers where you can add layers of a neural network one at a time.

#32 = number of convolution filters/kernels which learn the features of the image
#stride is just the shifting of the kernel accross the elements of the input matrix
#kernel size is the size of the filter. Could be just a single number or a tuple representing the matrix size.
#Activation is usually used on the output of the convulution layers and the dense layers.
#kernels are small weight matrices and the weights are learnable parametres
#We apply .product between the input image and the kernels(sum of the products of elementwise values)
#Input shape is only for the first convulutional layer
#remember, all these are layers being added to the neural network
#Convulution
emotion_model.add(Conv2D(32, kernel_size=(3,3), strides=1, activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2))) #downsampling to reduce size of the feature map to reduce the computational complexity and 2x2 are the dimensions of the pooling window. The max pooling operation takes the maximum value from each 2x2 window in the input
emotion_model.add(Dropout(0.5))#regularization to prevent overfitting. This means a random 50% of all the neurons are going to be set to zero each update. Prevents coadaption while introducing randomness into the network.

#You need to increase the number of features to learn more complex patterns
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))



In [10]:
#You flatten before you set up the densse neural connections
#Flatten usually has no input arguments
#1024 is the number of neurons in the dense layer
#Dropout is included but not Maxpooling 
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))

#last layer with 7 neurons and the softmax activation
emotion_model.add(Dense(7, activation='softmax'))

## 3. Defining the Learning Rate Schedule, Optimizers and Loss Functions



In [17]:
"""The purpose of using a learning rate schedule is to adapt the learning rate during training. Starting with a
higher learning rate can help the model converge faster in the initial stages, while gradually decreasing the 
learning rate can help fine-tune the model and avoid overshooting the optimal weights.
"""
learning_rate_schedule=tf.keras.optimizers.schedules.ExponentialDecay(
initial_learning_rate=0.0001,
decay_rate=0.9,
decay_steps=10000)#This parameter defines how often the learning rate will decay. After the specified number of steps (iterations or batches), the learning rate will be adjusted based on the decay rate.)
optimizer=Adam(learning_rate=learning_rate_schedule)

    #.compile method configure the learning process before training by setting the loss function, optimizer and the metrics to be monitored during training
emotion_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [18]:
#The learned parameters should be saved to this file for use later on
emotion_model.save_weights('model.h5')


## 4. Real Time Emotion Detection


In [20]:
# Disable OpenCL usage in cv2
cv2.ocl.setUseOpenCL(False)

# Define emotion labels
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

# Open a video capture object using the default camera (0)
cap = cv2.VideoCapture(0)

# Main video processing loop
while True:
    # Read a frame from the video capture
    success, frame = cap.read()

    # Break the loop if reading fails
    if not success:
        break

    # Convert the frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Load the qpre-trained face cascade classifier
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Detect faces in the grayscale frame
    num_faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    # Process each detected face
    for (x, y, w, h) in num_faces:
        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)

        # Extract the region of interest (ROI) and preprocess for emotion prediction
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

        # Predict the emotion using the pre-trained model
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))

        # Display the predicted emotion label on the frame
        cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # Display the processed video frame
    cv2.imshow('Video', cv2.resize(frame, (1200, 860), interpolation=cv2.INTER_CUBIC))

    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 22ms/step


KeyboardInterrupt: 

# THINGS I HAVE LEARNED FROM THIS: TAKEAWAYS

- Always divide the whole thing into smaller components i.e. importing module and preprocessing, build the network, 
learning parameters and optimization methods, real time object detection using opencv2 and the model you built
- For the model, you start out with a relatively simple architecture, of 2 Conv2D layers, MaxPooling2D and Drop out, then later on, you could add more convulutional layers with more filters to get the complex patterns and then set up the dense network for probabilities later on
- Specify the learning rate schedule as a decay, works better for convergence